# Import Required Libraries
Import the necessary libraries, such as requests, for making API calls.

In [2]:
# Import the necessary library for making API calls
# Install necessary libraries
'''%pip install requests
%pip install pandas
%pip install datetime
%pip install cudf
%pip install cupy
%pip install openpyxl
%pip install textblob
%pip install transformers
%pip install torch
%pip install import-ipynb
%pip install scikit-learn'''
# Import libraries
import import_ipynb
from Input_Tools import *

import requests
import pandas as pd
import datetime as dt
import numpy as np
import os as os
from textblob import TextBlob
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import matplotlib.pyplot as mdates
from sklearn.preprocessing import StandardScaler

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Define API Key
Define the API key

In [3]:
API_Eodhd = "66c0aeb1357b15.87356825"

#Important link: https://eodhd.com/financial-academy/financial-faq/fundamentals-glossary-common-stock


# Running correlation between all the stocks in the exchange
Creates a list of all the correlations betweens stocks and sorts them

In [ ]:
def calculate_and_sort_correlations(tickers_df, exchange, instrument_type="Common Stock", api_token=API_Eodhd, start_date="2025-01-01", end_date="2025-03-30", period="d"):
    """
    Calculate the correlation between all stocks in an exchange and sort them from least to highest.

    Parameters:
        tickers_df (pd.DataFrame): DataFrame containing stock tickers.
        exchange (str): Exchange code to append to tickers (e.g., "US").
        instrument_type (str): Type of the instrument to filter (e.g., "Common Stock").
        api_token (str): API token for authentication.
        start_date (str): Start date for historical data.
        end_date (str): End date for historical data.
        period (str): Period for historical data (e.g., "d" for daily).

    Returns:
        pd.DataFrame: DataFrame containing stock pairs and their correlation, sorted by correlation value.
    """
    # Filter tickers to include only the specified instrument type
    filtered_tickers_df = tickers_df[tickers_df["Type"] == instrument_type]

    stock_data = {}
    for ticker in filtered_tickers_df['Code']:
        try:
            # Combine ticker with exchange code
            full_ticker = f"{ticker}.{exchange}"
            stock_df = Fetch_historical_price(full_ticker, start_date, end_date, period, api_token, columns=["adjusted_close"])
            stock_data[ticker] = stock_df.set_index("date")["adjusted_close"]
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    # Combine all stock data into a single DataFrame
    combined_df = pd.DataFrame(stock_data)

    # Calculate correlation matrix
    correlation_matrix = combined_df.corr()

    # Create a list of all pairs and their correlations
    correlations = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            stock1 = correlation_matrix.columns[i]
            stock2 = correlation_matrix.columns[j]
            correlation = correlation_matrix.iloc[i, j]
            correlations.append({"Instrument 1": stock1, "Instrument 2": stock2, "Correlation": correlation})

    # Convert the list of correlations to a DataFrame
    correlations_df = pd.DataFrame(correlations)

    # Sort the DataFrame by correlation value
    correlations_df = correlations_df.sort_values(by="Correlation", ascending=True).reset_index(drop=True)

    return correlations_df

# Example usage
tickers = fetch_exchange_symbols("AS")
tickers_subset = tickers
sorted_correlations_df = calculate_and_sort_correlations(tickers_subset, "AS", instrument_type="ETF")
print(sorted_correlations_df)

In [ ]:
#optimized version

import pandas as pd
import numpy as np


def fetch_stock_data(ticker, exchange, start_date, end_date, period, api_token):
    """Helper function to fetch stock data for a single ticker."""
    try:
        full_ticker = f"{ticker}.{exchange}"
        stock_df = Fetch_historical_price(full_ticker, start_date, end_date, period, api_token, columns=["adjusted_close"])
        return ticker, stock_df.set_index("date")["adjusted_close"]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return ticker, None

def calculate_and_sort_correlations(tickers_df, exchange, instrument_type="Common Stock", api_token=API_Eodhd, start_date="2025-01-01", end_date="2025-03-30", period="d"):
    """
    Calculate the correlation between all stocks in an exchange and sort them from least to highest.

    Parameters:
        tickers_df (pd.DataFrame): DataFrame containing stock tickers.
        exchange (str): Exchange code to append to tickers (e.g., "US").
        instrument_type (str): Type of the instrument to filter (e.g., "Common Stock").
        api_token (str): API token for authentication.
        start_date (str): Start date for historical data.
        end_date (str): End date for historical data.
        period (str): Period for historical data (e.g., "d" for daily).

    Returns:
        pd.DataFrame: DataFrame containing stock pairs and their correlation, sorted by correlation value.
    """
    # Filter tickers to include only the specified instrument type
    filtered_tickers_df = tickers_df[tickers_df["Type"] == instrument_type]

    # Fetch stock data in parallel
    stock_data = {}
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(fetch_stock_data, ticker, exchange, start_date, end_date, period, api_token)
            for ticker in filtered_tickers_df['Code']
        ]
        for future in futures:
            ticker, data = future.result()
            if data is not None:
                stock_data[ticker] = data

    # Combine all stock data into a single DataFrame
    combined_df = pd.DataFrame(stock_data).dropna(axis=1, how="any")  # Drop columns with NaN values

    # Calculate correlation matrix
    correlation_matrix = combined_df.corr()

    # Flatten the correlation matrix into a DataFrame
    correlations_df = (
        correlation_matrix.stack()
        .reset_index()
        .rename(columns={"level_0": "Instrument 1", "level_1": "Instrument 2", 0: "Correlation"})
    )

    # Remove self-correlations and duplicates
    correlations_df = correlations_df[correlations_df["Instrument 1"] < correlations_df["Instrument 2"]]

    # Sort the DataFrame by correlation value
    correlations_df = correlations_df.sort_values(by="Correlation", ascending=True).reset_index(drop=True)

    return correlations_df

# Example usage
tickers = fetch_exchange_symbols("AS")
tickers_subset = tickers
sorted_correlations_df = calculate_and_sort_correlations(tickers_subset, "AS", instrument_type="ETF")
print(sorted_correlations_df)

# Exporting output to excel
Creates an excel export in the python folder

In [ ]:
# Define the output directory and file path
output_dir = r"E:\Business_NL\Python\Excel output"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
output_file = os.path.join(output_dir, "sorted_correlations.xlsx")

# Export the DataFrame to an Excel file
sorted_correlations_df.to_excel(output_file, index=False)

print(f"Sorted correlations exported to {output_file}")

# Analyzing sentement with finbert model
Creates a function to use finbert model to analyze sentement

In [ ]:

def analyze_sentiment_with_finbert(news_df):
    """
    Analyze the sentiment of news articles using FinBERT.

    Parameters:
        news_df (pd.DataFrame): DataFrame containing news articles.

    Returns:
        pd.DataFrame: DataFrame with an additional 'Sentiment' column.
    """
    if 'content' not in news_df.columns:
        raise ValueError("The DataFrame must contain a 'content' column for sentiment analysis.")
    
    # Load the FinBERT sentiment analysis pipeline with explicit truncation
    finbert = pipeline(
        "sentiment-analysis",
        model="yiyanghkust/finbert-tone",
        tokenizer="yiyanghkust/finbert-tone",
        device=0,  # Use CPU (-1) or GPU (0 or higher)
        truncation=True,
        max_length=512  # Explicitly set the maximum length
    )
    
    # Apply sentiment analysis to the 'content' column
    def analyze_text(text):
        try:
            return finbert(text[:512])[0]['label']  # Truncate text to 512 characters
        except Exception as e:
            return f"Error: {e}"
    
    news_df['Sentiment'] = news_df['content'].apply(analyze_text)
    
    return news_df

# Example usage
api_token = API_Eodhd  # Replace with your actual API token
stock = "AAPL.US"
begin_date = "2023-01-01"
end_date = "2025-03-01"
tag = "balance sheet"

# Fetch news data
news_df = fetch_news_data(stock, tag, begin_date, end_date, api_token=api_token)

# Perform sentiment analysis with FinBERT
news_with_sentiment = analyze_sentiment_with_finbert(news_df)

print(news_with_sentiment)

# Trading strategy: Use last 12 months dividends to price a stock/option
A trading strategy where I use last 12 months dividends and price the stock/option at 6% dividend yield

In [ ]:
def Fetch_dividend_share(tickers_df, exchange):
    """
    Fetch dividend share for all stocks in the tickers_df and combine them into a single DataFrame.

    Parameters:
        tickers_df (list or pd.Series): List of stock tickers (e.g., ["INGA", "ASML"]).
        exchange (str): Exchange code (e.g., "AS").

    Returns:
        pd.DataFrame: A DataFrame with two columns: 'Ticker' and 'DividendYield(%)'.
    """
    combined_data = []  # Initialize an empty list to store data
    full_tickers = [f"{ticker}.{exchange}" for ticker in tickers_df]

    for stock in full_tickers:
        try:
            # Fetch the dividend share for the current stock
            url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={API_Eodhd}&filter=Highlights::DividendShare&fmt=json'
            dividend_share = requests.get(url).json()

                   
            # Append the stock and dividend yield to the list
            combined_data.append({"Ticker": stock, "Dividendshare": dividend_share})
        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")

    # Convert the combined data into a DataFrame
    combined_df = pd.DataFrame(combined_data)

    # Drop rows with missing dividend yield values
    combined_df = combined_df.dropna(subset=["Dividendshare"])

    return combined_df

def Fetch_dividend_yield(tickers_df):
    """
    Fetch dividend share for all stocks in the tickers_df and combine them into a single DataFrame.

    Parameters:
        tickers_df (list or pd.Series): List of stock tickers (e.g., ["INGA.AS", "ASML.AS"]).

    Returns:
        pd.DataFrame: A DataFrame with two columns: 'Ticker' and 'Dividendshare'.
    """
    combined_data = []  # Initialize an empty list to store data

    for stock in tickers_df:
        try:
            # Fetch the dividend share for the current stock
            url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={API_Eodhd}&filter=Highlights::DividendYield&fmt=json'
            dividend_yield = requests.get(url).json()

            # Append the stock and dividend yield to the list
            combined_data.append({"Ticker": stock, "Dividend_yield": dividend_yield*100})
        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")

    # Convert the combined data into a DataFrame
    combined_df = pd.DataFrame(combined_data)

    # Drop rows with missing dividend yield values
    combined_df = combined_df.dropna(subset=["Dividend_yield"])

    return combined_df

In [ ]:
# Creating a theoretical value of the stock
tickers = ["INGA", "ASML", "ABN"]
Exchange = "AS"
Dividend_share = Fetch_dividend_share(tickers, Exchange).dropna(subset=["Dividendshare"])

Theoretical_DY = 0.06

# Ensure 'Dividendshare' is numeric, replace non-numeric values with 0
Dividend_share["Dividendshare"] = pd.to_numeric(Dividend_share["Dividendshare"], errors='coerce').fillna(0)

# Calculate the theoretical stock price, handling division by zero
Dividend_share["TheoreticalStockPrice"] = np.where(
    Theoretical_DY != 0,  # Check if the denominator is not zero
    Dividend_share["Dividendshare"] / Theoretical_DY,  # Perform the division
    0  # Return 0 if the denominator is zero
)

# Initialize a dictionary to store the 'High' prices for each ticker
high_prices = {}

# Loop through each ticker to fetch the latest price data and extract the 'High' value
for ticker in tickers:
    try:
        # Fetch the latest price data
        latest_price = fetch_intraday_data(f"{ticker}.{Exchange}").iloc[0]
        
        # Extract the 'High' value and store it in the dictionary
        high_prices[f"{ticker}.{Exchange}"] = latest_price["high"]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        high_prices[f"{ticker}.{Exchange}"] = None

# Add the 'High' values to the Dividend_share DataFrame
Dividend_share["StockPrice"] = Dividend_share["Ticker"].map(high_prices)

# Calculating upside/downside potential, handling division by zero
Dividend_share["Potential"] = np.where(
    Dividend_share["TheoreticalStockPrice"] != 0,  # Check if the denominator is not zero
    (Dividend_share["TheoreticalStockPrice"] - Dividend_share["StockPrice"]) * 100 / Dividend_share["TheoreticalStockPrice"],  # Perform the calculation
    0  # Return 0 if the denominator is zero
)

# Round the specified columns to 2 decimal places
Dividend_share["TheoreticalStockPrice"] = Dividend_share["TheoreticalStockPrice"].round(2)
Dividend_share["StockPrice"] = Dividend_share["StockPrice"].round(2)
Dividend_share["Potential"] = Dividend_share["Potential"].round(2)

# Print the updated DataFrame
print(Dividend_share)

In [ ]:
# finding the best stocks according to this model
#small caps
Small_Caps_list = ["PBH", "PORF", "LVIDE", "VALUE", "NSE", "CTAC", "BEVER", "NEDSE", "ECT"]
Exchange = "AS"
Dividend_share = Fetch_dividend_share(Small_Caps_list, Exchange).dropna(subset=["Dividendshare"])

Theoretical_DY = 0.06

# Ensure 'Dividendshare' is numeric, replace non-numeric values with 0
Dividend_share["Dividendshare"] = pd.to_numeric(Dividend_share["Dividendshare"], errors='coerce').fillna(0)

# Calculate the theoretical stock price, handling division by zero
Dividend_share["TheoreticalStockPrice"] = np.where(
    Theoretical_DY != 0,  # Check if the denominator is not zero
    Dividend_share["Dividendshare"] / Theoretical_DY,  # Perform the division
    0  # Return 0 if the denominator is zero
)

# Initialize a dictionary to store the 'High' prices for each ticker
high_prices = {}

# Loop through each ticker to fetch the latest price data and extract the 'High' value
for ticker in Small_Caps_list:
    try:
        # Fetch the latest price data
        latest_price = fetch_intraday_data(f"{ticker}.{Exchange}").iloc[0]
        
        # Extract the 'High' value and store it in the dictionary
        high_prices[f"{ticker}.{Exchange}"] = latest_price["high"]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        high_prices[f"{ticker}.{Exchange}"] = None

# Add the 'High' values to the Dividend_share DataFrame
Dividend_share["StockPrice"] = Dividend_share["Ticker"].map(high_prices)

# Calculating upside/downside potential, handling division by zero
Dividend_share["Potential"] = np.where(
    Dividend_share["TheoreticalStockPrice"] != 0,  # Check if the denominator is not zero
    (Dividend_share["TheoreticalStockPrice"] - Dividend_share["StockPrice"]) * 100 / Dividend_share["TheoreticalStockPrice"],  # Perform the calculation
    0  # Return 0 if the denominator is zero
)

# Round the specified columns to 2 decimal places
Dividend_share["TheoreticalStockPrice"] = Dividend_share["TheoreticalStockPrice"].round(2)
Dividend_share["StockPrice"] = Dividend_share["StockPrice"].round(2)
Dividend_share["Potential"] = Dividend_share["Potential"].round(2)

# Define the output directory and file path
output_dir = r"E:\Business NL\Python\Excel output"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
output_file = os.path.join(output_dir, "sorted_correlations.xlsx")

# Export the DataFrame to an Excel file
Dividend_share.to_excel(output_file, index=False)

print(f"Sorted correlations exported to {output_file}")

#Mid-caps
Mid_Caps_List = ["HYDRA", "KENDR", "SLIGR", "AJAX", "NEDAP", "PHARM","ACOMO", "TOM2","AMG", "PNL","AXS","BRNL","WHA","FFARM","NSI","SIFG","BSGR","ALFEN",
                 "ENVI","FAST","CMCOM","NXFIL","AZRN","TLT"]

Dividend_share = Fetch_dividend_share(Mid_Caps_List, Exchange).dropna(subset=["Dividendshare"])

Theoretical_DY = 0.06

# Ensure 'Dividendshare' is numeric, replace non-numeric values with 0
Dividend_share["Dividendshare"] = pd.to_numeric(Dividend_share["Dividendshare"], errors='coerce').fillna(0)

# Calculate the theoretical stock price, handling division by zero
Dividend_share["TheoreticalStockPrice"] = np.where(
    Theoretical_DY != 0,  # Check if the denominator is not zero
    Dividend_share["Dividendshare"] / Theoretical_DY,  # Perform the division
    0  # Return 0 if the denominator is zero
)

# Initialize a dictionary to store the 'High' prices for each ticker
high_prices = {}

# Loop through each ticker to fetch the latest price data and extract the 'High' value
for ticker in Mid_Caps_List:
    try:
        # Fetch the latest price data
        latest_price = fetch_intraday_data(f"{ticker}.{Exchange}").iloc[0]
        
        # Extract the 'High' value and store it in the dictionary
        high_prices[f"{ticker}.{Exchange}"] = latest_price["high"]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        high_prices[f"{ticker}.{Exchange}"] = None

# Add the 'High' values to the Dividend_share DataFrame
Dividend_share["StockPrice"] = Dividend_share["Ticker"].map(high_prices)

# Calculating upside/downside potential, handling division by zero
Dividend_share["Potential"] = np.where(
    Dividend_share["TheoreticalStockPrice"] != 0,  # Check if the denominator is not zero
    (Dividend_share["TheoreticalStockPrice"] - Dividend_share["StockPrice"]) * 100 / Dividend_share["TheoreticalStockPrice"],  # Perform the calculation
    0  # Return 0 if the denominator is zero
)

# Round the specified columns to 2 decimal places
Dividend_share["TheoreticalStockPrice"] = Dividend_share["TheoreticalStockPrice"].round(2)
Dividend_share["StockPrice"] = Dividend_share["StockPrice"].round(2)
Dividend_share["Potential"] = Dividend_share["Potential"].round(2)

print(Dividend_share)

# Large caps
Large_Caps_List = ["KPN","MT","OCI","WKL","APAM","BAMNB","PHIA","RAND","CRBN","HEIO","ASM","AKZA","REN","AALB","HAL","HEIA","AGN","ASML","SBMO","BESI",
                   "SHELL","VPK","TFG","TWEKA","FUR","UNA","INGA","AD","ABN","ARCAD","ASRNL","BFIT","ECMPA","FLOW","GLPG","HEIJM","IMCD","NRP","NN","PSH",
                   "LIGHT","TKWY","VLK","ADYEN","PRX","CCEP","REINA","JDEP","INPST","CTPNV","ALLFG","UMG","EXO","DSFIR","FER","THEON","CVC"]

Dividend_share = Fetch_dividend_share(Mid_Caps_List, Exchange).dropna(subset=["Dividendshare"])

Theoretical_DY = 0.06

# Ensure 'Dividendshare' is numeric, replace non-numeric values with 0
Dividend_share["Dividendshare"] = pd.to_numeric(Dividend_share["Dividendshare"], errors='coerce').fillna(0)

# Calculate the theoretical stock price, handling division by zero
Dividend_share["TheoreticalStockPrice"] = np.where(
    Theoretical_DY != 0,  # Check if the denominator is not zero
    Dividend_share["Dividendshare"] / Theoretical_DY,  # Perform the division
    0  # Return 0 if the denominator is zero
)

# Initialize a dictionary to store the 'High' prices for each ticker
high_prices = {}

# Loop through each ticker to fetch the latest price data and extract the 'High' value
for ticker in Mid_Caps_List:
    try:
        # Fetch the latest price data
        latest_price = fetch_intraday_data(f"{ticker}.{Exchange}").iloc[0]
        
        # Extract the 'High' value and store it in the dictionary
        high_prices[f"{ticker}.{Exchange}"] = latest_price["high"]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        high_prices[f"{ticker}.{Exchange}"] = None

# Add the 'High' values to the Dividend_share DataFrame
Dividend_share["StockPrice"] = Dividend_share["Ticker"].map(high_prices)

# Calculating upside/downside potential, handling division by zero
Dividend_share["Potential"] = np.where(
    Dividend_share["TheoreticalStockPrice"] != 0,  # Check if the denominator is not zero
    (Dividend_share["TheoreticalStockPrice"] - Dividend_share["StockPrice"]) * 100 / Dividend_share["TheoreticalStockPrice"],  # Perform the calculation
    0  # Return 0 if the denominator is zero
)

# Round the specified columns to 2 decimal places
Dividend_share["TheoreticalStockPrice"] = Dividend_share["TheoreticalStockPrice"].round(2)
Dividend_share["StockPrice"] = Dividend_share["StockPrice"].round(2)
Dividend_share["Potential"] = Dividend_share["Potential"].round(2)

print(Dividend_share)


In [ ]:
# Define the output directory and file path
output_dir = r"E:\Business_NL\Python\Excel output"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
output_file = os.path.join(output_dir, "dividend_shares.xlsx")

# Create a dictionary to store DataFrames for each category
dividend_data = {}

# Small Caps
Small_Caps_list = ["PBH", "PORF", "LVIDE", "VALUE", "NSE", "CTAC", "BEVER", "NEDSE", "ECT"]
Dividend_share_small = Fetch_dividend_share(Small_Caps_list, Exchange).dropna(subset=["Dividendshare"])
Dividend_share_small["Dividendshare"] = pd.to_numeric(Dividend_share_small["Dividendshare"], errors='coerce').fillna(0)
Dividend_share_small["TheoreticalStockPrice"] = np.where(
    Theoretical_DY != 0,
    Dividend_share_small["Dividendshare"] / Theoretical_DY,
    0
)

# Calculate StockPrice and Potential for Small Caps
high_prices = {}
for ticker in Small_Caps_list:
    try:
        latest_price = fetch_intraday_data(f"{ticker}.{Exchange}").iloc[0]
        high_prices[f"{ticker}.{Exchange}"] = latest_price["high"]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        high_prices[f"{ticker}.{Exchange}"] = None

Dividend_share_small["StockPrice"] = Dividend_share_small["Ticker"].map(high_prices)
Dividend_share_small["Potential"] = np.where(
    Dividend_share_small["TheoreticalStockPrice"] != 0,
    (Dividend_share_small["TheoreticalStockPrice"] - Dividend_share_small["StockPrice"]) * 100 / Dividend_share_small["TheoreticalStockPrice"],
    0
)
Dividend_share_small = Dividend_share_small.round(2)
dividend_data["Small Caps"] = Dividend_share_small

# Mid Caps
Mid_Caps_List = ["HYDRA", "KENDR", "SLIGR", "AJAX", "NEDAP", "PHARM", "ACOMO", "TOM2", "AMG", "PNL", "AXS", "BRNL", "WHA", "FFARM", "NSI", "SIFG", "BSGR", "ALFEN", "ENVI", "FAST", "CMCOM", "NXFIL", "AZRN", "TLT"]
Dividend_share_mid = Fetch_dividend_share(Mid_Caps_List, Exchange).dropna(subset=["Dividendshare"])
Dividend_share_mid["Dividendshare"] = pd.to_numeric(Dividend_share_mid["Dividendshare"], errors='coerce').fillna(0)
Dividend_share_mid["TheoreticalStockPrice"] = np.where(
    Theoretical_DY != 0,
    Dividend_share_mid["Dividendshare"] / Theoretical_DY,
    0
)

# Calculate StockPrice and Potential for Mid Caps
high_prices = {}
for ticker in Mid_Caps_List:
    try:
        latest_price = fetch_intraday_data(f"{ticker}.{Exchange}").iloc[0]
        high_prices[f"{ticker}.{Exchange}"] = latest_price["high"]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        high_prices[f"{ticker}.{Exchange}"] = None

Dividend_share_mid["StockPrice"] = Dividend_share_mid["Ticker"].map(high_prices)
Dividend_share_mid["Potential"] = np.where(
    Dividend_share_mid["TheoreticalStockPrice"] != 0,
    (Dividend_share_mid["TheoreticalStockPrice"] - Dividend_share_mid["StockPrice"]) * 100 / Dividend_share_mid["TheoreticalStockPrice"],
    0
)
Dividend_share_mid = Dividend_share_mid.round(2)
dividend_data["Mid Caps"] = Dividend_share_mid

# Large Caps
Large_Caps_List = ["KPN", "MT", "OCI", "WKL", "APAM", "BAMNB", "PHIA", "RAND", "CRBN", "HEIO", "ASM", "AKZA", "REN", "AALB", "HAL", "HEIA", "AGN", "ASML", "SBMO", "BESI", "SHELL", "VPK", "TFG", "TWEKA", "FUR", "UNA", "INGA", "AD", "ABN", "ARCAD", "ASRNL", "BFIT", "ECMPA", "FLOW", "GLPG", "HEIJM", "IMCD", "NRP", "NN", "PSH", "LIGHT", "TKWY", "VLK", "ADYEN", "PRX", "CCEP", "REINA", "JDEP", "INPST", "CTPNV", "ALLFG", "UMG", "EXO", "DSFIR", "FER", "THEON", "CVC"]
Dividend_share_large = Fetch_dividend_share(Large_Caps_List, Exchange).dropna(subset=["Dividendshare"])
Dividend_share_large["Dividendshare"] = pd.to_numeric(Dividend_share_large["Dividendshare"], errors='coerce').fillna(0)
Dividend_share_large["TheoreticalStockPrice"] = np.where(
    Theoretical_DY != 0,
    Dividend_share_large["Dividendshare"] / Theoretical_DY,
    0
)

# Calculate StockPrice and Potential for Large Caps
high_prices = {}
for ticker in Large_Caps_List:
    try:
        latest_price = fetch_intraday_data(f"{ticker}.{Exchange}").iloc[0]
        high_prices[f"{ticker}.{Exchange}"] = latest_price["high"]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        high_prices[f"{ticker}.{Exchange}"] = None

Dividend_share_large["StockPrice"] = Dividend_share_large["Ticker"].map(high_prices)
Dividend_share_large["Potential"] = np.where(
    Dividend_share_large["TheoreticalStockPrice"] != 0,
    (Dividend_share_large["TheoreticalStockPrice"] - Dividend_share_large["StockPrice"]) * 100 / Dividend_share_large["TheoreticalStockPrice"],
    0
)
Dividend_share_large = Dividend_share_large.round(2)
dividend_data["Large Caps"] = Dividend_share_large

# Export all DataFrames to a single Excel file with separate sheets
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, df in dividend_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Dividend shares exported to {output_file}")

# Separating the stocks into 3 quintiles
Separating the stocks into small cap, midcap and large cap

In [ ]:
# calculate the daily trading market volume = close price * volume
# Take an average of the last 30 days
# separate them into three quintiles

# Plotting rolling correlation between two stocks
Understand the rolling correlation between two stocks

In [ ]:
import matplotlib.dates as mdates 
# Calculate the start date for the last 3 years
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=3 * 365)


# Define stock details
stock1 = "INGA"
exchange1 = "AS"
stock2 = "ABN"
exchange2 = "AS"
period = "d"

# Function to fetch historical prices and plot rolling correlation
def plot_rolling_correlation(stock1, exchange1, stock2, exchange2, start_date, end_date, window=90):
    # Fetch historical prices for both stocks
    stock1_df = Fetch_historical_price(stock1, exchange1, start_date, end_date, period, API_Eodhd)
    stock2_df = Fetch_historical_price(stock2, exchange2, start_date, end_date, period, API_Eodhd)

    # Ensure both DataFrames have a 'date' column and convert it to datetime
    stock1_df['date'] = pd.to_datetime(stock1_df['date'])
    stock2_df['date'] = pd.to_datetime(stock2_df['date'])

    # Sort the DataFrames by date and set the 'date' column as the index
    stock1_df = stock1_df.sort_values(by='date').set_index('date')
    stock2_df = stock2_df.sort_values(by='date').set_index('date')

    # Align the two DataFrames on their dates
    combined_df = pd.concat([stock1_df['adjusted_close'], stock2_df['adjusted_close']], axis=1, keys=['Stock1', 'Stock2']).dropna()
    
    # Calculate daily returns for both stocks
    combined_df['Stock1_Returns'] = combined_df['Stock1'].pct_change()
    combined_df['Stock2_Returns'] = combined_df['Stock2'].pct_change()

    # Calculate rolling correlation
    rolling_corr = combined_df['Stock1'].rolling(window=window).corr(combined_df['Stock2'])

    # Plot the rolling correlation with color based on value
    plt.figure(figsize=(10, 6))
    for i in range(1, len(rolling_corr)):
        if rolling_corr[i] > 0:
            plt.plot(rolling_corr.index[i-1:i+1], rolling_corr[i-1:i+1], color='red')
        else:
            plt.plot(rolling_corr.index[i-1:i+1], rolling_corr[i-1:i+1], color='green')

    plt.title(f'{window}-Day Rolling Correlation Between {stock1} and {stock2}')
    plt.xlabel('Date')
    plt.ylabel('Correlation')
    plt.grid()

    # Format the date axis
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Show every 12 months
    plt.gcf().autofmt_xdate()  # Rotate date labels for better readability

    plt.show()

# Call the function to plot the rolling correlation
plot_rolling_correlation(stock1, exchange1, stock2, exchange2, start_date, end_date, window=90)

In [ ]:
#plotting multiple rolling correlations

plot_rolling_correlation("INGA", "AS", "AAPL", "US", start_date, end_date, window=90)
plot_rolling_correlation("INGA", "AS", "IUSA", "AS", start_date, end_date, window=90)

# Creating a 4 factor model
Factor 1 - Correlation with portfolio
Factor 2 - Dividend yield
Factor 3 - Price momentum (55 days)
Factor 4 - Price momentum (25 days)
Equal weighting to all factors

In [4]:
# Creating my portfolio

# Define the path to the Excel file

excel_file_path_portfolio = r"E:\Business_NL\Python\Portfolio_list.xlsx"

# Read column C from the Excel file into a pandas DataFrame
# Assuming column C contains the data you need and has a header
portfolio_list_stock = pd.read_excel(excel_file_path_portfolio, usecols="C")
portfolio_list_quantity = pd.read_excel(excel_file_path_portfolio, usecols="D")

# Define the start and end dates
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(weeks=52*7)

# Getting portfolio information
portfolio_stocks = portfolio_list_stock["Stock_name"]
portfolio_quantity = portfolio_list_quantity["Quantity"]
Margin_amount = 16000

# Initializing the portfolio DataFrame
portfolio_init = pd.DataFrame({
    "Stock": portfolio_stocks,
    "Quantity": portfolio_quantity
})



# Initialize a DataFrame to store daily portfolio values
daily_portfolio_values = pd.DataFrame()

# Loop through each ticker in the portfolio
for index, row in portfolio_init.iterrows():
    ticker = row["Stock"]
    quantity = row["Quantity"]
    try:
        # Fetch historical price data (adjusted close prices)
        stock_data = Fetch_historical_price(
            f"{ticker}", start_date, end_date, period="d", columns=["adjusted_close"]
        )
        
        # Add a column for the stock's daily value (price * quantity)
        stock_data["Daily_Value"] = stock_data["adjusted_close"] * quantity
        
        # Add the stock's daily value to the portfolio DataFrame
        if daily_portfolio_values.empty:
            daily_portfolio_values = stock_data[["date", "Daily_Value"]].rename(columns={"Daily_Value": ticker})
        else:
            daily_portfolio_values = daily_portfolio_values.merge(
                stock_data[["date", "Daily_Value"]].rename(columns={"Daily_Value": ticker}),
                on="date",
                how="outer"
            )
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Calculate the total portfolio value for each day
daily_portfolio_values["Total_Portfolio_Value"] = daily_portfolio_values.drop(columns=["date"]).sum(axis=1) - Margin_amount

# Add the margin amount as a column
daily_portfolio_values["Margin_Amount"] = Margin_amount

# Drop rows with NaN values
daily_portfolio_values = daily_portfolio_values.dropna()

# Display the daily portfolio values
#print(daily_portfolio_values)

In [ ]:
def calculate_four_factor_score(stock):
    """
    Calculate the four factors for a stock:
    1. Spearman correlation with portfolio daily values
    2. Dividend yield
    3. Price momentum (55 days)
    4. Price momentum (25 days)
    Returns: correlation, dividend_yield, momentum_55, momentum_25 (all rounded to 3 decimals)
    """
    try:
        # Calculate log returns for Total_Portfolio_Value
        daily_portfolio_values['log_return'] = np.log(
            daily_portfolio_values['Total_Portfolio_Value'] / daily_portfolio_values['Total_Portfolio_Value'].shift(1)
        )
        daily_portfolio_returns = daily_portfolio_values.dropna()
        end_date = dt.datetime.now()
        start_date = end_date - dt.timedelta(days=90)

        # Factor 1: Spearman correlation with portfolio
        def process_stock(stock):
            try:
                stock_data = Fetch_historical_price(stock, start_date, end_date, period="d", columns=["adjusted_close"])
                stock_data['log_return'] = np.log(stock_data['adjusted_close'] / stock_data['adjusted_close'].shift(1))
                stock_data = stock_data.dropna()
                merged_data = pd.merge(
                    daily_portfolio_returns[['date', 'log_return']],
                    stock_data[['date', 'log_return']],
                    on='date',
                    suffixes=('_portfolio', f'_{stock}')
                )
                correlation = merged_data['log_return_portfolio'].corr(merged_data[f'log_return_{stock}'], method='spearman')
                return correlation if correlation is not None else 0
            except Exception as e:
                print(f"Error processing {stock}: {e}")
                return 0  # If error, treat correlation as 0

        correlation = process_stock(stock)

        # Factor 2: Dividend yield
        dividend_yield = 0
        try:
            dividend_yield_df = Fetch_dividend_yield([stock])
            if not dividend_yield_df.empty and "Dividend_yield" in dividend_yield_df.columns:
                value = dividend_yield_df.loc[dividend_yield_df['Ticker'] == stock, "Dividend_yield"]
                if not value.empty and pd.notna(value.values[0]):
                    dividend_yield = float(value.values[0])
        except Exception as e:
            print(f"Error fetching dividend yield for {stock}: {e}")
            dividend_yield = 0  # Explicitly set to zero on any error

        # Factor 3: Price momentum (55 days)
        try:
            momentum_55 = fetch_price_momentum(stock, 55)
            if momentum_55 is None or pd.isna(momentum_55):
                momentum_55 = 0
        except Exception:
            momentum_55 = 0

        # Factor 4: Price momentum (25 days)
        try:
            momentum_25 = fetch_price_momentum(stock, 25)
            if momentum_25 is None or pd.isna(momentum_25):
                momentum_25 = 0
        except Exception:
            momentum_25 = 0

        # Round all outputs to 3 decimals
        return (
            round(correlation, 3),
            round(dividend_yield, 3),
            round(momentum_55, 3),
            round(momentum_25, 3)
        )

    except Exception as e:
        print(f"Error calculating four-factor values for {stock}: {e}")
        return None, None, None, None

# Example usage
stock = "ABN.AS"
start_date = "2023-01-01"
end_date = "2023-12-31"

correlation, dividend_yield, momentum_55, momentum_25 = calculate_four_factor_score(stock)
print(f"  Factor 1 (Correlation): {correlation}")
print(f"  Factor 2 (Dividend Yield): {dividend_yield}")
print(f"  Factor 3 (Momentum 55): {momentum_55}")
print(f"  Factor 4 (Momentum 25): {momentum_25}")

In [ ]:
# Define the list of exchanges
exchange_list = ["AS","PA","US"]

# Initialize an empty list to store DataFrames
combined_data = []

# Loop through each exchange and fetch the data
for exchange in exchange_list:
    df = fetch_stocks_by_market_cap_and_exchange(100000000000, exchange)
    combined_data.append(df)

# Combine all DataFrames into a single DataFrame
stock_list = pd.concat(combined_data, ignore_index=True)

# Display the combined DataFrame
print(stock_list)



In [ ]:
# Define start and end dates for the analysis
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=90)

# Function to wrap the four-factor calculation for parallel execution
def run_four_factor(stock):
    # Returns: (correlation, dividend_yield, momentum_55, momentum_25)
    return (stock, *calculate_four_factor_score(stock))

# Run in parallel for all stocks in stock_list["Stock_name"]
with ThreadPoolExecutor() as executor:
    results = list(executor.map(run_four_factor, stock_list["Stock_name"]))

# Convert results to DataFrame with factor titles (no Raw_Score column)
four_factor_df = pd.DataFrame(
    results,
    columns=[
        "Stock",
        "Correlation_with_Portfolio",
        "Dividend_Yield",
        "Momentum_55d",
        "Momentum_25d"
    ]
)

# Drop rows with None values (if any)
four_factor_df = four_factor_df.dropna(subset=["Correlation_with_Portfolio", "Dividend_Yield", "Momentum_55d", "Momentum_25d"])

# Normalize each factor to the range [-1, 1]
factor_columns = ["Correlation_with_Portfolio", "Dividend_Yield", "Momentum_55d", "Momentum_25d"]
scaler = MinMaxScaler(feature_range=(-1, 1))
four_factor_df[[f + "_Norm" for f in factor_columns]] = scaler.fit_transform(four_factor_df[factor_columns])

# Specify coefficients for each factor
correlation_coef = -0.25
dividend_yield_coef = 0.25
momentum_55_coef = 0.25
momentum_25_coef = 0.25

# Calculate normalized score using the specified coefficients
four_factor_df["Normalized_Score"] = (
    correlation_coef * four_factor_df["Correlation_with_Portfolio_Norm"] +
    dividend_yield_coef * four_factor_df["Dividend_Yield_Norm"] +
    momentum_55_coef * four_factor_df["Momentum_55d_Norm"] +
    momentum_25_coef * four_factor_df["Momentum_25d_Norm"]
).round(3)

# Sort by highest normalized score
four_factor_df = four_factor_df.sort_values(by="Normalized_Score", ascending=False).reset_index(drop=True)

print(four_factor_df[[
    "Stock",
    "Normalized_Score",
    "Correlation_with_Portfolio_Norm",
    "Dividend_Yield_Norm",
    "Momentum_55d_Norm",
    "Momentum_25d_Norm"
]].head(10))  # Display top 10 stocks

In [ ]:
# Output four_factor_df to Excel
output_dir = r"E:\Business_NL\Python\Excel output"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "four_factor_scores.xlsx")

four_factor_df.to_excel(output_file, index=False)
print(f"Four-factor scores exported to {output_file}")


# Creating a logistic regression model
The model will try to predict if the value after a month will be higher or lower

In [ ]:
# Define the start and end dates
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(weeks=52*7)
# Retriving stock price
ABN_price = Fetch_historical_price("ABN.AS", start_date, end_date, period="d", columns=["adjusted_close","volume"])
KBW_price = Fetch_historical_price("KBWB.US", start_date, end_date, period="d", columns=["adjusted_close","volume"])
daily_portfolio_values = daily_portfolio_values.dropna()


# Merging data into a single Dataframe

# Ensure 'date' columns are datetime for proper merging
ABN_price['date'] = pd.to_datetime(ABN_price['date'])
KBW_price['date'] = pd.to_datetime(KBW_price['date'])
daily_portfolio_values['date'] = pd.to_datetime(daily_portfolio_values['date'])

# Merge ABN_price and KBW_price on 'date'
merged_df = pd.merge(ABN_price, KBW_price, on='date', suffixes=('_ABN', '_KBW'))


# Merge only the 'Total_Portfolio_Value' column from daily_portfolio_values
Combined_df = pd.merge(
    merged_df,
    daily_portfolio_values[['date', 'Total_Portfolio_Value']],
    on='date',
    how='inner'
)



# Columns for log returns
log_return_cols = ['adjusted_close_ABN', 'adjusted_close_KBW', 'Total_Portfolio_Value']

# Columns to keep as absolute values (exclude date and log return columns)
abs_cols = [col for col in Combined_df.columns if col not in log_return_cols + ['date']]

# Calculate log returns for selected columns
log_returns = np.log(Combined_df[log_return_cols] / Combined_df[log_return_cols].shift(1))

# Get absolute values for other columns (shift to align with log returns)
abs_values = Combined_df[abs_cols].iloc[1:].reset_index(drop=True)

# Align log_returns index with abs_values
log_returns = log_returns.iloc[1:].reset_index(drop=True)

# Add the date column back (skip the first row to match log_returns)
date_col = Combined_df['date'].iloc[1:].reset_index(drop=True)

# Combine all into a single DataFrame
Combined_df_log = pd.concat([date_col, log_returns, abs_values], axis=1)

# Reorder columns to make 'date' the first column
cols = Combined_df_log.columns.tolist()
cols = ['date'] + [col for col in cols if col != 'date']
Combined_df_log = Combined_df_log[cols]

print(Combined_df_log.head())

        date  adjusted_close_ABN  adjusted_close_KBW  Total_Portfolio_Value  \
0 2018-06-19           -0.000449            0.000540              -0.013941   
1 2018-06-20            0.012123           -0.000900               0.016043   
2 2018-06-21           -0.014839            0.000360              -0.024951   
3 2018-06-22            0.025045           -0.010132              -0.001301   
4 2018-06-25           -0.015125           -0.011704              -0.095628   

   volume_ABN  volume_KBW  
0     2897670      114661  
1     2244582      139888  
2     2336461      236573  
3     1995351      855619  
4     1101921      215020  


In [4]:

end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(weeks=52*5)
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")
ABN_news = Fetch_news_data("ABN.AS", start_str, end_str, offset=0, api_token=API_Eodhd)
# Filter only 'date' and 'content' columns
ABN_news = ABN_news[['date', 'content']]

print(ABN_news)

                          date  \
0    2025-05-22T14:09:23+00:00   
1    2025-05-15T07:04:26+00:00   
2    2025-05-15T05:31:45+00:00   
3    2025-05-14T05:14:00+00:00   
4    2025-05-09T13:07:23+00:00   
..                         ...   
300  2020-08-12T07:05:45+00:00   
301  2020-08-12T05:00:00+00:00   
302  2020-07-24T11:00:45+00:00   
303  2020-07-24T11:00:45+00:00   
304  2020-06-17T06:00:01+00:00   

                                               content  
0    The Dutch government has lowered its stake in ...  
1    Net Profit: EUR619 million. Return on Equity: ...  
2    As European markets navigate a landscape marke...  
3    ABN AMRO\n\nABN AMRO Bank posts net profit of ...  
4    The latest developments around tariffs remain ...  
..                                                 ...  
300  By Bart H. Meijer\n\nAMSTERDAM (Reuters) - ABN...  
301  ABN AMRO reports net loss of EUR 5 million for...  
302  AMSTERDAM (Reuters) - Dutch bank ABN Amro on F...  
303  AMSTERDAM (Reute

In [23]:
# ...existing code...
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(weeks=52*5)
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")
ABN_news = Fetch_news_data("ABN.AS", start_str, end_str, offset=0, api_token=API_Eodhd)
# Filter only 'date' and 'content' columns
ABN_news = ABN_news[['date','sentiment']]

# Convert 'date' to just date (YYYY-MM-DD)
ABN_news['date'] = pd.to_datetime(ABN_news['date']).dt.date

# Extract 'polarity' from the 'sentiment' dictionary in ABN_news

# If 'sentiment' is a string representation of a dict, use ast.literal_eval to convert
import ast

def extract_polarity(sentiment):
    if isinstance(sentiment, dict):
        return sentiment.get('polarity', None)
    try:
        sentiment_dict = ast.literal_eval(sentiment)
        return sentiment_dict.get('polarity', None)
    except Exception:
        return None

ABN_news['polarity'] = ABN_news['sentiment'].apply(extract_polarity)

print(ABN_news[['date', 'polarity']].head())


# Group by date and take the mean polarity for each date (handles multiple news per day)
polarity_by_date = (
    ABN_news.groupby('date')['polarity']
    .mean()
    .reset_index()
)
# ...existing code...

         date  polarity
0  2025-05-22     0.977
1  2025-05-15     0.997
2  2025-05-15     0.997
3  2025-05-14     1.000
4  2025-05-09     1.000


In [23]:

end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(weeks=52*5)
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")
ABN_news = Fetch_news_data("ABN.AS", start_str, end_str, offset=0, api_token=API_Eodhd)
# Filter only 'date' and 'content' columns
ABN_news = ABN_news[['date', 'content']]

# Convert 'date' to just date (YYYY-MM-DD)
ABN_news['date'] = pd.to_datetime(ABN_news['date']).dt.date

from transformers import pipeline
from sklearn.preprocessing import MinMaxScaler

# Use FinBERT to analyze sentiment for each news content
finbert = pipeline(
    "sentiment-analysis",
    model="yiyanghkust/finbert-tone",
    tokenizer="yiyanghkust/finbert-tone",
    device=-1,  # Use -1 for CPU
    truncation=True,
    max_length=512
)

def get_finbert_score(text):
    try:
        output = finbert(str(text)[:512])
        return output[0]['score']
    except Exception as e:
        print(f"Error: {e}")
        return None

# Ensure 'date' is datetime for grouping, and remove time part
ABN_news["date"] = pd.to_datetime(ABN_news["date"]).dt.date

# Apply FinBERT sentiment scoring (actual score, not mapped)
ABN_news["finbert_score"] = ABN_news["content"].apply(get_finbert_score)

# Group by date and calculate average FinBERT sentiment score for each day
daily_sentiment = (
    ABN_news.groupby("date")["finbert_score"]
    .mean()
    .reset_index()
    .rename(columns={"finbert_score": "avg_finbert_score"})
)

print("Device set to use cpu")
print(daily_sentiment.head(10))


Device set to use cpu


Device set to use cpu
         date  avg_finbert_score
0  2020-06-17           0.999940
1  2020-07-24           0.998417
2  2020-08-12           0.999953
3  2020-08-24           0.999963
4  2020-09-16           0.988015
5  2020-11-11           0.999999
6  2020-11-17           0.999109
7  2020-11-30           0.999961
8  2021-02-10           0.999589
9  2021-03-02           0.999975


In [ ]:
# Merge daily_sentiment with Combined_df_log on 'date', fill missing avg_finbert_score with 0
# Ensure both 'date' columns are datetime.date type for proper merging
Combined_df_log['date'] = pd.to_datetime(Combined_df_log['date']).dt.date
daily_sentiment['date'] = pd.to_datetime(daily_sentiment['date']).dt.date

# Merge on 'date'
Combined_df_log_with_sentiment = pd.merge(
    Combined_df_log,
    daily_sentiment,
    on='date',
    how='left'
)

# Fill missing avg_finbert_score with 0
Combined_df_log_with_sentiment['avg_finbert_score'] = Combined_df_log_with_sentiment['avg_finbert_score'].fillna(0)



         date  adjusted_close_ABN  volume_ABN  adjusted_close_KBW  volume_KBW  \
0  2020-06-16            0.069609    0.137662            0.022273    0.107321   
1  2020-06-17           -0.016265    0.054480           -0.023741   -0.514224   
2  2020-06-18           -0.016788   -0.321381            0.000000   -0.045133   
3  2020-06-19           -0.023012    0.510204           -0.006396    0.328715   
4  2020-06-22           -0.008578   -1.036782           -0.006940   -0.467136   

   adjusted_close_JPM  volume_JPM  Total_Portfolio_Value  avg_finbert_score  
0            0.007968    0.001804               0.098782            0.00000  
1           -0.025604   -0.349985               0.009881            0.99994  
2           -0.005443    0.059096              -0.010762            0.00000  
3           -0.011486    0.889202               0.022180            0.00000  
4           -0.010897   -0.912063              -0.034371            0.00000  


In [29]:
#Splitting a combined DataFrame into training and test sets based on date

# Ensure date columns are datetime.date for merging
Combined_df_log['date'] = pd.to_datetime(Combined_df_log['date']).dt.date
# Ensure polarity_by_date is defined before this cell!
# Remove any existing 'polarity', 'polarity_x', or 'polarity_y' columns to avoid merge errors
for col in ['polarity', 'polarity_x', 'polarity_y']:
    if col in Combined_df_log.columns:
        Combined_df_log = Combined_df_log.drop(columns=[col])

polarity_by_date['date'] = pd.to_datetime(polarity_by_date['date']).dt.date

# Merge with Combined_df_log
Combined_df_log = pd.merge(
    Combined_df_log,
    polarity_by_date,
    on='date',
    how='left'
)

# Fill missing polarity values with 0
Combined_df_log['polarity'] = Combined_df_log['polarity'].fillna(0)



# Ensure 'date' column is datetime
Combined_df_log['date'] = pd.to_datetime(Combined_df_log['date'])

# Sort by date just in case
Combined_df_log = Combined_df_log.sort_values('date').reset_index(drop=True)

# Get the minimum date
min_date = Combined_df_log['date'].min()

# Define training and test period
train_end_date = min_date + pd.DateOffset(years=6)
test_end_date = train_end_date + pd.DateOffset(years=2)

# Split the data
train_df = Combined_df_log[(Combined_df_log['date'] >= min_date) & (Combined_df_log['date'] < train_end_date)].reset_index(drop=True)
test_df = Combined_df_log[(Combined_df_log['date'] >= train_end_date) & (Combined_df_log['date'] < test_end_date)].reset_index(drop=True)

print(test_df.head())


        date  adjusted_close_ABN  adjusted_close_KBW  Total_Portfolio_Value  \
0 2024-06-20            0.006464            0.000757               0.008653   
1 2024-06-21           -0.004522           -0.005695              -0.004168   
2 2024-06-24            0.009031            0.016328              -0.000115   
3 2024-06-25           -0.010973           -0.011581              -0.002785   
4 2024-06-26           -0.007818           -0.003635               0.001586   

   volume_ABN  volume_KBW  polarity  
0     1900763      918926  0.000000  
1     4111381      668936  0.968500  
2     2232462     1325620  0.498500  
3     1487734      341121  0.999000  
4     1997544      447869  0.332333  


In [30]:


from sklearn.linear_model import LogisticRegression
import numpy as np

# Prepare the data for logistic regression
# We want to predict if adjusted_close_ABN will be positive 1 month (about 21 trading days) from now

# Create the target variable: 1 if adjusted_close_ABN is positive 21 days ahead, else 0
train_df = train_df.copy()
train_df['target_1m'] = (train_df['adjusted_close_ABN'].shift(-21) > 0).astype(int)

print (train_df.tail(5))

# Drop the last 21 rows where the target is NaN
train_df = train_df.dropna(subset=['target_1m'])

# Select all predictor columns except 'date', 'target_1m', and the target itself
exclude_cols = ['date', 'target_1m']
if 'adjusted_close_ABN' in train_df.columns:
    exclude_cols.append('adjusted_close_ABN')
X = train_df.drop(columns=exclude_cols).values
y = train_df['target_1m'].values

# Fit logistic regression
logreg = LogisticRegression(max_iter=100000)
logreg.fit(X, y)

# Predict probability of positive value 1 month from now for each row in train_df
train_df['prob_positive_1m'] = logreg.predict_proba(X)[:, 1]





           date  adjusted_close_ABN  adjusted_close_KBW  \
1492 2024-06-12            0.014042            0.013485   
1493 2024-06-13           -0.020485           -0.005372   
1494 2024-06-14           -0.014008           -0.005596   
1495 2024-06-17            0.004580            0.011542   
1496 2024-06-18            0.006187            0.009326   

      Total_Portfolio_Value  volume_ABN  volume_KBW  polarity  target_1m  
1492               0.005851     2517999      517204     0.998          0  
1493               0.006439     2918350      328389     0.998          0  
1494               0.008006     3793421      319084     0.000          0  
1495               0.003340     2625417      410230     0.000          0  
1496               0.005006     1784772      550588     0.000          0  


In [31]:


from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Use the same exclude_cols as in training
exclude_cols = ['date', 'target_1m']
if 'adjusted_close_ABN' in train_df.columns:
    exclude_cols.append('adjusted_close_ABN')

# Remove 'prob_positive_1m' from features for both train and test
if 'prob_positive_1m' in train_df.columns:
    exclude_cols.append('prob_positive_1m')

# Prepare test data (use same predictors as training)
test_df = test_df.copy()
test_df['target_1m'] = (test_df['adjusted_close_ABN'].shift(-21) > 0).astype(int)
test_df = test_df.dropna(subset=['target_1m'])

# Ensure columns match training data (order and names)
feature_cols = [col for col in train_df.columns if col not in exclude_cols]
# Only use columns that exist in test_df to avoid KeyError
feature_cols_test = [col for col in feature_cols if col in test_df.columns]

# Prepare X and y for train and test
X_train = train_df[feature_cols].values
y_train = train_df['target_1m'].values
X_test = test_df[feature_cols_test].values
y_test = test_df['target_1m'].values

# Retrain logistic regression with correct features
logreg = LogisticRegression(max_iter=100000)
logreg.fit(X_train, y_train)

# Predict probabilities and classes
test_df['prob_positive_1m'] = logreg.predict_proba(X_test)[:, 1]
test_df['predicted_class'] = (test_df['prob_positive_1m'] >= 0.5).astype(int)

# Evaluate model
accuracy = accuracy_score(y_test, test_df['predicted_class'])
roc_auc = roc_auc_score(y_test, test_df['prob_positive_1m'])
report = classification_report(y_test, test_df['predicted_class'])

print(f"Test Accuracy: {accuracy:.3f}")
print(f"Test ROC AUC: {roc_auc:.3f}")
print("Classification Report:\n", report)
print(test_df[['date', 'prob_positive_1m', 'predicted_class', 'target_1m']].head())

# Show variable importance (absolute value of coefficients)
coefs = logreg.coef_[0]
importance = pd.Series(coefs, index=feature_cols).sort_values(key=abs, ascending=False)
print("\nVariable importance (absolute value of logistic regression coefficients):")
print(importance)



Test Accuracy: 0.473
Test ROC AUC: 0.468
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.50      0.49       120
           1       0.47      0.45      0.46       119

    accuracy                           0.47       239
   macro avg       0.47      0.47      0.47       239
weighted avg       0.47      0.47      0.47       239

        date  prob_positive_1m  predicted_class  target_1m
0 2024-06-20          0.497487                0          1
1 2024-06-21          0.507212                1          1
2 2024-06-24          0.494686                0          0
3 2024-06-25          0.501660                1          0
4 2024-06-26          0.502326                1          1

Variable importance (absolute value of logistic regression coefficients):
volume_KBW              -3.836499e-08
volume_ABN               1.325882e-08
polarity                -9.194089e-13
adjusted_close_KBW       6.383277e-15
Total_Portfolio_Value    1.